*General hints:* <br>
* You may use another notebook to test different approaches and ideas. When complete and mature, turn your code snippets into the requested functions in this notebook for submission. 
* Make sure the function implementations are generic and can be applied to any dataset (not just the one provided).
* Add explanatory code comments in the code cells. Make sure that these comments improve our understanding of your implementation decisions.

-----
* Create a variable holding your student id, as shown below. 
* Simply replace the example (`01234567`) with your actual student id having a total of 8 digits. 
* Maintain the variable as a string, do NOT change its type in this notebook!
* *Note: If your student id has 7 digits, add a leading 0. The final student id MUST have 8 digits!*

In [59]:
mn = '12213171'

## 0. Import

Implement a function `tidy` which imports the data set assigned and provided to you as a CSV file into a `pandas` dataframe. Access the data set and establish whether your data set is tidy. If not, clean the data set before continuing with Step 1. Mind all rules of tidying data sets in this step. Make sure you comply to the following statements:
* If there is an index column (row numbers) in your tidied dataset, keep it.
* The following columns, once identified, correspond to variables 1:1 (no need for transformations):
  * `full_name`
  * `automotive`
  * `color`
  * `job`
  * `address`
  * `coordinates`
* The tidied dataset should have a total of 8 columns (not including the index), the first column should be `full_name`.
* Mind the intended content of each attribute (e.g. full_name should contain the full name of a person, no need to change that)
* If tidy or done, have the function `tidy` return the ready data set as a dataframe.

Note that `tidy` must take a single parameter that holds the basename of the CSV file (i.e., the name without file extension). Do NOT change the name of the file, do not overwrite the original data file, and make sure you submit your final ZIP following the [Code of Conduct](https://datascience.ai.wu.ac.at/ws21/dataprocessing1/code_of_conduct.html) requirements. Especially, make sure you put your data file in a folder called `data/` when submitting.

In [72]:
import pandas as pd
def tidy(x):
    filePath = f"./data/{x}.csv"
    data = pd.read_csv(filePath)

    # change rows and columns
    dataTranspose = data.transpose() # https://www.geeksforgeeks.org/python-pandas-dataframe-transpose/
    # select columns and put it in first row
    dataTranspose.columns = dataTranspose.iloc[0] # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html
    # delet first row because we do not want two rows with colum names
    dataTranspose = dataTranspose.drop(dataTranspose.index[0])

    # split the colum date_time/full_company_name in seperate columns
    # https://www.delftstack.com/de/howto/python-pandas/split-column-in-python-pandas/
    if 'date_time/full_company_name' in dataTranspose.columns:
        dataTranspose[['date_time', 'full_company_name']] = dataTranspose['date_time/full_company_name'].str.split(pat="\.", n=1, expand=True)

    # delete colum date_time/full_company_name   
    dataTranspose = dataTranspose.drop(columns=['date_time/full_company_name'])
    return pd.DataFrame(dataTranspose)

print(tidy(mn))

Unnamed: 0        full_name automotive            color  \
0               Janice Mann   TZK-5163       DarkViolet   
1            Alexandra Haas    KIY 037       PowderBlue   
2              Jason Arroyo    U09 4JK             Snow   
3            Alejandro Hart    W19 9QR        GoldenRod   
4             Timothy Baker   40Q 8824           Orange   
...                     ...        ...              ...   
1619         Tracy Erickson    190-684            Green   
1620        Lindsay Osborne    257 ZR8         DarkCyan   
1621            John Barton     399GLG  MediumVioletRed   
1622            Kayla Russo    88C D74        LightGray   
1623              Sara Hall    SPI7199             Lime   

Unnamed: 0                            job             address  \
0              Designer, fashion/clothing        Melissahaven   
1               Building control surveyor      Rodriguezburgh   
2                  Chief Strategy Officer         Gardnerberg   
3                              

In [61]:
from nose.tools import assert_equal
import pandas
assert_equal(type(tidy(mn)), pandas.core.frame.DataFrame)
assert_equal(len((tidy(mn)).columns), 8)
assert_equal(list((tidy(mn)).columns)[0], "full_name")


-------
## 1. Missing values

### 1.1 Code part
Implement a function called `missing_values` which takes as an input a dataframe and check if there are any missing values in the dataset. Record the row ids of the observations containing missing values as a list of numbers and make sure that the function returns the recorded list in the end. If there are no missing values, `missing_values` should return an empty list.

NOTE: Try to find out how missing values are incoded in your datasest and which missing values occur in your dataset by manual inspection, but at least test for the following: `"nan"`,`"NA"`,`"-inf"`,`"inf"`,`"None"`; also treat fields containing the numeric value 0 as well as for empty fields and fields containing only white spaces as missing. (We are aware that this test generic test might be overshooting in practice ;-))

In [62]:
def missing_values(x):
    # replace all missing symbols with NA
    missing_symbols = ["nan", "NA", "-inf", "inf", "None", "0", "", "—", "--"]
    x.replace(missing_symbols, pd.NA, inplace=True)
    
    # It searches in every row if there is any missing value. If there is a missing value, it indicates its index and added it to a list
    missingRows = x.isna().any(axis=1) # https://pandas.pydata.org/docs/reference/api/pandas.isna.html#pandas.isna
    missingRowsList = []
    for i in missingRows.index:
        if missingRows[i]:
            missingRowsList.append(int(i))
    return missingRowsList

missing_values(tidy(mn))

[9,
 31,
 63,
 71,
 93,
 118,
 150,
 177,
 221,
 344,
 361,
 369,
 461,
 512,
 579,
 595,
 603,
 706,
 718,
 804,
 833,
 913,
 967,
 969,
 1010,
 1029,
 1037,
 1061,
 1078,
 1102,
 1111,
 1143,
 1227,
 1229,
 1251,
 1258,
 1411,
 1441,
 1454,
 1529,
 1598]

In [63]:
from nose.tools import assert_equal
assert_equal(type(missing_values(tidy(mn))), list)
assert_equal(all(isinstance(i, int) for i in missing_values(tidy(mn))), True)


### 1.2. Analytical part

* Does the dataset contain missing values?
* If no, explain how you proved that this is actually the case.
* If yes, describe the discovered missing values. What could be an explanation for their missingness?

Write your answer in the markdown cell bellow. Do NOT delete or replace the answer cell with another one!


YOUR ANSWER HERE
Yes, the data set contains missing values. These missing values are over 41 rows distributed. There could be several issues for the missingness:
    - there could be variables that are not applicable to all objects or some variables are not available at the time of recording
    - there could also be some missing values because of errors in data-generation process (MCAR, MAR, MNAR)


------
## 2. Handling missing values
### 2.1. Code part
Apply a (simple) function called *handling_missing_values* for handling missing values using an adequate single-imputation technique  of your choice per type of missing values. Make use of the techniques learned in Unit 4. The function should take as an input a dataframe and return the updated dataframe. Mind the following:
- The objective is to apply single imputation on these synthetic data. Do not make up a background story (at this point)!
- Do NOT simply drop the missing values. This is not an option.
- The imputation technique must be adequate for a given variable type (quantitative, qualitative). 

In [64]:
def handling_missing_values(x):
    data = x
    missing_symbols = ["nan", "NA", "-inf", "inf", "None", "0", "", "—", "--"]
    # Replace all missing_symbols in NA
    for i in missing_symbols:
        data.replace(missing_symbols, pd.NA, inplace=True)
    
    # If i is numeric, replace missing value with mean
    for i in data.columns:
        if data[i].dtype in ['int', 'float']:
            mean = data[i].mean()
            data[i].fillna(mean, inplace=True) # fillna detects missing values: https://pandas.pydata.org/docs/reference/api/pandas.isna.html#pandas.isna
        # if i is not numeric, replace value with mode
        else:
            mode = data[i].mode()[0]
            data[i].fillna(mode, inplace=True) # https://pandas.pydata.org/docs/reference/api/pandas.isna.html#pandas.isna
    
    return data

handling_missing_values(tidy(mn)) 

Unnamed: 0,full_name,automotive,color,job,address,coordinates,date_time,full_company_name
0,Janice Mann,TZK-5163,DarkViolet,"Designer, fashion/clothing",Melissahaven,"(Decimal('52.183991'), Decimal('67.055665'))",2005-08-27 02:25:16,"886684Arroyo, Brewer and Smith"
1,Alexandra Haas,KIY 037,PowderBlue,Building control surveyor,Rodriguezburgh,"(Decimal('-20.4303945'), Decimal('-75.895260'))",2004-11-28 18:38:06,"943086Diaz, Simmons and Robertson"
2,Jason Arroyo,U09 4JK,Snow,Chief Strategy Officer,Gardnerberg,"(Decimal('-51.0134155'), Decimal('107.324399'))",2010-03-31 17:21:06,"308270Fisher, Robinson and Jones"
3,Alejandro Hart,W19 9QR,GoldenRod,Geochemist,Tranland,"(Decimal('78.0828945'), Decimal('-33.302858'))",1995-02-07 13:54:19,"641452Nguyen, Wright and Taylor"
4,Timothy Baker,40Q 8824,Orange,Network engineer,South Nataliemouth,"(Decimal('81.551225'), Decimal('-3.628981'))",2010-11-15 03:10:44,"467221Williams, Fox and Thomas"
...,...,...,...,...,...,...,...,...
1619,Tracy Erickson,190-684,Green,Haematologist,Lake Alanhaven,"(Decimal('73.2160295'), Decimal('-59.039948'))",1997-09-09 03:35:02,548491Cuevas-Beltran
1620,Lindsay Osborne,257 ZR8,DarkCyan,Hydrogeologist,Ibarraton,"(Decimal('23.153734'), Decimal('35.590832'))",2000-04-19 17:31:04,"619146Parker, Green and Humphrey"
1621,John Barton,399GLG,MediumVioletRed,Mudlogger,East Roberttown,"(Decimal('-15.993773'), Decimal('158.303118'))",2002-06-12 03:52:45,064890Russell-Torres
1622,Kayla Russo,88C D74,LightGray,Operational investment banker,Erikfurt,"(Decimal('-37.1624925'), Decimal('-153.002489'))",2014-08-16 17:33:50,428063Mitchell Ltd


In [65]:
from nose.tools import assert_equal
assert_equal(len(missing_values(handling_missing_values(tidy(mn)))), 0)
assert_equal(handling_missing_values(tidy(mn)).shape, tidy(mn).shape)

### 2.2. Analytical part
Discuss the implications. Answer the following:

- How would you qualify the data-generating processes leading to different types of missing values, provided that the data was not synthetic?
- What are the benefits and disadvantages of the chosen single-imputation technique?
- How would you apply a multiple-imputation technique to one type of missing values, if applicable at all?
- We asked you to test for/treat as missing values by checking certain field values, as well as empty fields or fields containing the numeric value 0... what are potential problems of this heuristics?

Write your answer in the markdown cell bellow. Do NOT delete or replace the answer cell with another one!

YOUR ANSWER HERE
For each different type of missing values we have to handle the missingness different. Missing Not at Random is the most challenging type.
Benefits: They are easy to implement.
Disadvantage: They do not reflect the variance in ("real") datasets without missing values.
Multiple imputation technique is effective when the data is MCAR or MAR. At first we have to create multiple complete datasets. Missing values will be replaced with plausible values for example with predictive mean matching. Then the analysis is then performed for each imputed dataset. The combined statistic is then computed.
Potentials problems that comes up with replacing values that are 0 are that we do not know if these values are actual correct observed. 


-----
## 3. Duplicate entries
Implement a function called `duplicates` that takes as an input a (tidy) dataframe `x`. Assume that `duplicates` receives a dataframe as returned from your Step 0 implementation of `tidy`. It then checks whether there are any duplicates in the dataset. Record the row ids of the observations being duplicates and have `duplicates` returns the list in the end. An empty list indicates the absence of duplicated observations.

In [66]:
def duplicates(x):
    # get all dublicates as True
    duplicate = x.duplicated(keep=False) # https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html
    # return a list of dublicated index
    return x.index[duplicate].tolist()

print(duplicates(tidy(mn)))

['66', '67', '76', '77', '189', '190', '232', '233', '246', '247', '303', '304', '322', '323', '400', '401', '433', '434', '458', '459', '495', '496', '551', '552', '559', '560', '614', '615', '681', '682', '691', '692', '778', '779', '838', '839', '845', '846', '857', '858', '864', '865', '874', '875', '917', '918', '944', '945', '954', '955', '1130', '1131', '1134', '1135', '1210', '1211', '1241', '1242', '1261', '1262', '1305', '1306', '1310', '1311', '1317', '1318', '1353', '1354', '1366', '1367', '1375', '1376', '1382', '1383', '1438', '1439', '1487', '1488', '1567', '1568', '1582', '1583', '1600', '1601']


In [67]:
from nose.tools import assert_equal
assert_equal(type(duplicates(tidy(mn))), list)


-----
## 4. Handling duplicate entries
### 4.1. Code part
Implement a function called `handling_duplicate_entries` for handling duplicate entries. Again, the function is assumed to receive a tidied data set as obtained from Step 0. It deduplicates the tidy data set. The function then returns the dataframe without duplicates.

In [70]:
def handling_duplicate_entries(x):
    data = x
    
    # get sorted data list
    dataList = sorted(data.values.tolist(), key = str)
    
    newData = []
    
    #compare every column
    for i in range(len(dataList)):    
        if i == 0 or dataList[i] != dataList[i-1]:
            newData.append(dataList[i])
            
    return pd.DataFrame(newData, columns=data.columns)

handling_duplicate_entries(tidy(mn))

Unnamed: 0,full_name,automotive,color,job,address,coordinates,date_time,full_company_name
0,-inf,5CG8993,Aquamarine,Mining engineer,Patriciaberg,"(Decimal('-30.901764'), Decimal('130.963991'))",2021-08-02 03:46:44,619691Ball Group
1,-inf,865-WVK,Peru,Forensic scientist,Port Todd,"(Decimal('-23.7939645'), Decimal('-66.604842'))",1994-12-28 16:10:42,429847Johnson-Harris
2,Aaron Atkinson,648 8774,OldLace,Building control surveyor,Alvaradofort,"(Decimal('72.2648865'), Decimal('35.123363'))",2015-06-11 09:01:31,801119Daniels-Ward
3,Aaron Lopez,TAH 931,Gainsboro,Health and safety adviser,Michellemouth,"(Decimal('-19.7863615'), Decimal('136.337420'))",2015-02-08 17:05:46,533016Harris Inc
4,Abigail Bailey,440HNJ,DarkGray,Lexicographer,Port Gregory,"(Decimal('-56.2017425'), Decimal('-82.610075'))",2006-04-30 12:54:53,018433Wiggins-Cain
...,...,...,...,...,...,...,...,...
1577,Zachary Ward,961 1GN,LightGray,Careers information officer,Jasmineland,"(Decimal('-4.498669'), Decimal('-149.037765'))",2006-07-04 01:42:32,073461Dunn-Mann
1578,—,88A 5825,LightCoral,Herbalist,Marquezstad,"(Decimal('-47.481363'), Decimal('-134.215281'))",2001-11-13 07:48:04,"498836Jackson, Acosta and Moreno"
1579,NaN,3G 8848C,GreenYellow,Public affairs consultant,North Crystal,"(Decimal('-25.696378'), Decimal('50.191305'))",2004-07-14 11:49:37,"401208Sandoval, Henry and Zamora"
1580,NaN,4529,MistyRose,"Lighting technician, broadcasting/film/video",New Anthonyton,"(Decimal('47.670012'), Decimal('101.261321'))",2006-05-17 12:57:03,604746Johnston-Figueroa


In [71]:
from nose.tools import assert_equal
assert_equal(len(duplicates(handling_duplicate_entries(tidy(mn)))), 0)

### 4.2. Analytical part
Discuss the implications. 

- What are the benefits and disadvantages of the chosen duplicate definition and the chosen duplicate-handling technique?
- Name and explain one alternative definition of (intra-source) duplicates for the given dataset!

Write your answer in the markdown cell bellow. Do NOT delete or replace the answer cell with another one!

YOUR ANSWER HERE
It takes a lot of memory to store the entire dataset during sorting process, however it is very easy to implement and to understand the code.
Intra-source duplicates specifically refers to observations that are duplicated or repeated within the same dataset like in column 66. 